## Data Input

**Current Data Availability:**

- Monthly excess returns for **6 anomalies**.
- **Fama–French factors** (MKT, SMB, HML).
- **Regime Classification:**
  - **Pre-Crisis:** 2003-01 to 2007-11
  - **Crisis:** 2007-12 to 2009-06
  - **Post-Crisis:** 2009-07 to 2014-05

---

## Time-Series Regression Analysis (Revised)

This model evaluates whether the risk-adjusted returns of each anomaly changed significantly during the crisis regime, while also accounting for changes in factor exposures (risk).

### Model Specifications

We estimate the following specifications with interaction terms to allow factor loadings to vary by regime.

**Model 1: Pre-crisis vs. Crisis (Baseline: Pre-crisis)**
$$R_t = \alpha_1 + \gamma_1 \cdot \text{Crisis}_t + \beta_1 \cdot \text{Factors}_t + \delta_1 (\text{Crisis}_t \times \text{Factors}_t) + \epsilon_t$$

**Model 2: Crisis vs. Post-crisis (Baseline: Crisis)**
$$R_t = \alpha_2 + \gamma_2 \cdot \text{Post}_t + \beta_2 \cdot \text{Factors}_t + \delta_2 (\text{Post}_t \times \text{Factors}_t) + \epsilon_t$$

### Variable Definitions

- **$R_t$**: The monthly long-short anomaly return.
- **$\text{Factors}_t$**: Vector of Fama–French three factors (MKT, SMB, HML).
- **Dummy Variables**:
  - **$\text{Crisis}_t$**: 1 during the crisis period (Dec 2007–June 2009), 0 otherwise.
  - **$\text{Post}_t$**: 1 during the post-crisis period (July 2009–Dec 2014), 0 otherwise.
- **Coefficients**:
  - **$\beta_1, \beta_2$**: Factor loadings during the respective **baseline** periods (Pre-crisis for Model 1; Crisis for Model 2).
  - **$\gamma_1, \gamma_2$**: Captures the abnormal returns (shift in alpha) during the Crisis and Post-crisis periods, respectively.
  - **$\delta_1, \delta_2$**: Vectors of **interaction coefficients** capturing changes in factor loadings during the Crisis and Post-crisis periods.
- **$\epsilon_t$**: The error term.


In [ ]:
import pandas as pd
from IPython.display import display
import statsmodels.api as sm

In [ ]:
excess_returns = pd.read_excel('./Regression Data/excess_returns.xlsx', index_col=0)
display(round(excess_returns,3))
excess_returns.info()

In [ ]:
ff_factors = pd.read_excel('./Regression Data/fama_french_factors.xlsx', index_col='date')
display(round(ff_factors.head(),3))
print(ff_factors.info())

In [ ]:
# Merge Data
# We join the anomaly returns with the Fama-French factors.
# We only need Mkt-RF, SMB, and HML from the factors df.
df_combined = pd.merge(
    excess_returns, 
    ff_factors[['Mkt-RF', 'SMB', 'HML']], 
    left_index=True, 
    right_index=True, 
    how='inner'
)
display(round(df_combined.head(),3))

In [ ]:
# List of anomaly columns (Dependent Variables)
anomalies = ['Accruals', 'Asset Growth', 'BM', 'Gross Profit', 'Momentum', 'Leverage Ret']

In [ ]:
# --- Updated Regression Function with Interactions ---
def run_regressions_with_interactions(data, anomalies, dummy_col_name):
    results_list = []
    
    # Define Factor columns
    factors = ['Mkt-RF', 'SMB', 'HML']
    
    # Create Interaction Terms: Dummy * Factor
    # This captures the change in factor loading during the specific regime (delta coefficients)
    interaction_cols = []
    for f in factors:
        int_col = f"{dummy_col_name}_x_{f}"
        data[int_col] = data[dummy_col_name] * data[f]
        interaction_cols.append(int_col)
    
    # Independent Variables: Constant + Factors + Dummy + Interactions
    X_cols = factors + [dummy_col_name] + interaction_cols
    X = sm.add_constant(data[X_cols])
    
    for anomaly in anomalies:
        # Dependent Variable
        y = data[anomaly]
        
        # Fit OLS
        model = sm.OLS(y, X).fit(cov_type='HAC', cov_kwds={'maxlags':3})
        
        # Store results
        # We now capture the base betas (beta1/beta2) and the interaction deltas (delta1/delta2)
        results_list.append({
            'Anomaly': anomaly,
            'Alpha': model.params['const'],
            'Alpha_P_Value': model.pvalues['const'],
            
            # Regime Dummy Coefficient (Gamma)
            'Gamma_Coef': model.params[dummy_col_name],
            'Gamma_P_Value': model.pvalues[dummy_col_name],
            'Gamma_t_stat': model.tvalues[dummy_col_name],
            
            # Base Factor Loadings (Beta)
            'Base_Beta_MKT': model.params['Mkt-RF'],
            'Base_Beta_MKT_Pval': model.pvalues['Mkt-RF'],
            'Base_Beta_SMB': model.params['SMB'],
            'Base_Beta_SMB_Pval': model.pvalues['SMB'],
            'Base_Beta_HML': model.params['HML'],
            'Base_Beta_HML_Pval': model.pvalues['HML'],
            
            # Interaction Coefficients (Delta) - Change in loadings
            'Delta_MKT': model.params[f"{dummy_col_name}_x_Mkt-RF"],
            'Delta_MKT_Pval': model.pvalues[f"{dummy_col_name}_x_Mkt-RF"],
            'Delta_SMB': model.params[f"{dummy_col_name}_x_SMB"],
            'Delta_SMB_Pval': model.pvalues[f"{dummy_col_name}_x_SMB"],
            'Delta_HML': model.params[f"{dummy_col_name}_x_HML"],
            'Delta_HML_Pval': model.pvalues[f"{dummy_col_name}_x_HML"],
            
            'Adjusted R2': model.rsquared_adj,
            'Observations': int(model.nobs),
        })
        
    return pd.DataFrame(results_list)

In [ ]:
# --- Run Revised Model 1: Pre-Crisis vs. Crisis ---
# Specification: R = alpha + gamma*Crisis + beta*Factors + delta(Crisis*Factors) + e

# Filter: Keep only Pre-Crisis (Baseline) and Crisis rows
df_m1 = df_combined[df_combined['Regime'].isin(['Pre-Crisis', 'Crisis'])].copy()

# Dummy: 1 if Crisis, 0 if Pre-Crisis
df_m1['CrisisDummy'] = (df_m1['Regime'] == 'Crisis').astype(int)

print("Running Model 1 (Pre-Crisis vs. Crisis) with Interactions...")
results_m1 = run_regressions_with_interactions(df_m1, anomalies, 'CrisisDummy')

results_m1.to_excel('./Regression results with interaction term/model_1_revised_results.xlsx', index=False)
display(round(results_m1, 3))

In [ ]:
# --- Run Revised Model 2: Crisis vs. Post-Crisis ---
# Specification: R = alpha + gamma*Post + beta*Factors + delta(Post*Factors) + e

# Filter: Keep only Crisis (Baseline) and Post-Crisis rows
df_m2 = df_combined[df_combined['Regime'].isin(['Crisis', 'Post-Crisis'])].copy()

# Dummy: 1 if Post-Crisis, 0 if Crisis
df_m2['PostCrisisDummy'] = (df_m2['Regime'] == 'Post-Crisis').astype(int)

print("\nRunning Model 2 (Crisis vs. Post-Crisis) with Interactions...")
results_m2 = run_regressions_with_interactions(df_m2, anomalies, 'PostCrisisDummy')

results_m2.to_excel('./Regression results with interaction term/model_2_revised_results.xlsx', index=False)
display(round(results_m2, 3))